In [ ]:
from environs import env

from export_volumes_pipeline.pacs_client import PacsClient

env.read_env("../.env")

modality = "CT"
institution_name = "Neuroradiologie"

client = PacsClient(
    env("CALLING_AE_TITLE"),
    env("PACS_AE_TITLE"),
    env("PACS_HOST"),
    env.int("PACS_PORT"),
)

# InstitutionName and StationName are not supported in
# Bonn at STUDY level :-(
query = {
    "QueryRetrieveLevel": "STUDY",
    "StudyDate": "20250101",
    "ModalitiesInStudy": modality,
    "StudyInstanceUID": "",
    "StudyDescription": "",
    "AccessionNumber": "",
}

studies = list(client.find(query))

found_study = None
found_series = None
found_image = None
for study in studies:
    query = {
        "QueryRetrieveLevel": "SERIES",
        "StudyInstanceUID": study.StudyInstanceUID,
        "SeriesInstanceUID": "",
        "SeriesDescription": "",
        "Modality": modality,
    }

    series_list = list(client.find(query))

    # StationName always returns "UNKNOWN", but InstitutionName
    # is supported on image level.
    query = {
        "QueryRetrieveLevel": "IMAGE",
        "StudyInstanceUID": series_list[0].StudyInstanceUID,
        "SeriesInstanceUID": series_list[0].SeriesInstanceUID,
        "InstitutionName": "",
    }

    images = list(client.find(query))
    if images and institution_name in images[0].InstitutionName:
        found_study = study
        found_series = series_list[0]
        found_image = images[0]
        break

query = {
    "QueryRetrieveLevel": "SERIES",
    "StudyInstanceUID": found_image.StudyInstanceUID,
    "SeriesInstanceUID": found_image.SeriesInstanceUID,
}

datasets = client.retrieve(query)

print(found_study)
print("---")
print(found_series)
print("---")
print(found_image)
print("+++")
for ds in datasets:
    print(ds)
    print("---")
